# Table of contents
1. [Imports](#imports)
2. [Preprocessing](#preproc)
3. [Missing data imputation](#md)
4. [K means clustering](#kmeans)
5. [KNN](#knn)

## 1. Imports

In [1]:
import copy
import matplotlib.pyplot as plt
plt.style.use('classic')
import numpy as np
import pandas as pd
import seaborn as sns 
import statistics
%matplotlib inline 
from os import getcwd
import plotly.express as px
import matplotlib.cm as cm
import pylab as pl

from scipy import sparse
from scipy import stats
from scipy.sparse import csr_matrix

import sklearn
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


getcwd()

'C:\\Users\\tessa\\UVT Thesis\\Untitled Folder'

In [2]:
from surprise import accuracy
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms.knns import KNNBasic

## 2. Preprocessing

In [3]:
# run the data that was saved

In [5]:
data_outer = pd.read_csv('data_outer_preprocessed.csv')

In [ ]:
testdata = pd.read_csv('testdata_outer.csv')

# 3. Missing data imputation

In [ ]:
dataset_for_clustering = data_outer[['energy', 'danceability', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                                        'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature',
                                        'chorus_hit', 'sections']]

testdata_for_clustering = testdata[['energy', 'danceability', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                                        'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature',
                                        'chorus_hit', 'sections']]

In [ ]:
miceimputed = dataset_for_clustering.copy(deep=True)
mice_imputer = IterativeImputer()
miceimputed.iloc[:, :] = mice_imputer.fit_transform(dataset_for_clustering)

In [ ]:
testmiceimputed = testdata_for_clustering.copy(deep=True)
testmice_imputer = IterativeImputer()
testmiceimputed.iloc[:, :] = testmice_imputer.fit_transform(testdata_for_clustering)

In [ ]:
data_outer['danceability'] = miceimputed['danceability']
data_outer['energy'] = miceimputed['energy']
data_outer['key'] = miceimputed['key']
data_outer['loudness'] = miceimputed['loudness']
data_outer['mode'] = miceimputed['mode']
data_outer['speechiness'] = miceimputed['speechiness']
data_outer['acousticness'] = miceimputed['acousticness']
data_outer['instrumentalness'] = miceimputed['instrumentalness']
data_outer['liveness'] = miceimputed['liveness']
data_outer['valence'] = miceimputed['valence']
data_outer['tempo'] = miceimputed['tempo']
data_outer['duration_ms'] = miceimputed['duration_ms']
data_outer['time_signature'] = miceimputed['time_signature']
data_outer['chorus_hit'] = miceimputed['chorus_hit']
data_outer['sections'] = miceimputed['sections']

testdata['danceability'] = testmiceimputed['danceability']
testdata['energy'] = testmiceimputed['energy']
testdata['key'] = testmiceimputed['key']
testdata['loudness'] = testmiceimputed['loudness']
testdata['mode'] = testmiceimputed['mode']
testdata['speechiness'] = testmiceimputed['speechiness']
testdata['acousticness'] = testmiceimputed['acousticness']
testdata['instrumentalness'] = testmiceimputed['instrumentalness']
testdata['liveness'] = testmiceimputed['liveness']
testdata['valence'] = testmiceimputed['valence']
testdata['tempo'] = testmiceimputed['tempo']
testdata['duration_ms'] = testmiceimputed['duration_ms']
testdata['time_signature'] = testmiceimputed['time_signature']
testdata['chorus_hit'] = testmiceimputed['chorus_hit']
testdata['sections'] = testmiceimputed['sections']

In [ ]:
#data_outer.to_csv('imputed_data.csv')

In [ ]:
#testdata.to_csv('test_imputed_data.csv')

In [ ]:
# or load directly into python

In [6]:
data_outer = pd.read_csv('imputed_data.csv')

C:\Users\tessa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
testdata = pd.read_csv('test_imputed_data.csv')

In [ ]:
data_outer = data_outer.drop('Unnamed: 0', axis=1)
testdata = testdata.drop('Unnamed: 0', axis=1)

In [ ]:
data_outer.shape

# 3. K means clustering

In [8]:
dataset_for_clustering = data_outer[['energy', 'danceability', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                                        'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature',
                                        'chorus_hit', 'sections']]

testdata_for_clustering = testdata[['energy', 'danceability', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                                        'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature',
                                        'chorus_hit', 'sections']]

In [9]:
# then, we make the clusters
km = KMeans(n_clusters=8, init='k-means++', random_state=42)
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', km)],verbose=True)

song_cluster_pipeline.fit(dataset_for_clustering)
song_cluster_labels = song_cluster_pipeline.predict(dataset_for_clustering)
data_outer['cluster'] = song_cluster_labels

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   2.1s
[Pipeline] ............ (step 2 of 2) Processing kmeans, total= 1.2min


In [ ]:
data_outer['cluster'].value_counts()

In [10]:
cluster0 = data_outer.loc[data_outer['cluster'] == 0]
print(cluster0.shape)

cluster1 = data_outer.loc[data_outer['cluster'] == 1]
print(cluster1.shape)

cluster2 = data_outer.loc[data_outer['cluster'] == 2]
print(cluster2.shape)

cluster3 = data_outer.loc[data_outer['cluster'] == 3]
print(cluster3.shape)

cluster4 = data_outer.loc[data_outer['cluster'] == 4]
print(cluster4.shape)

cluster5 = data_outer.loc[data_outer['cluster'] == 5]
print(cluster5.shape)

cluster6 = data_outer.loc[data_outer['cluster'] == 6]
print(cluster6.shape)

cluster7 = data_outer.loc[data_outer['cluster'] == 7]
print(cluster7.shape)

(139236, 31)
(1590113, 31)
(40586, 31)
(20963, 31)
(46329, 31)
(8497, 31)
(70850, 31)
(170372, 31)


#### Important test set and divide clusters

In [11]:
#test_song_cluster_labels = song_cluster_pipeline.fit(testdata_for_clustering)
test_song_cluster_labels = song_cluster_pipeline.predict(testdata_for_clustering)

# assign these clusters to the testdata set
testdata['cluster'] = test_song_cluster_labels

In [ ]:
testdata['cluster'].value_counts()

In [12]:
# test clusters
testcluster0 = testdata.loc[testdata['cluster'] == 0]
print(testcluster0.shape)

testcluster1 = testdata.loc[testdata['cluster'] == 1]
print(testcluster1.shape)

testcluster2 = testdata.loc[testdata['cluster'] == 2]
print(testcluster2.shape)

testcluster3 = testdata.loc[testdata['cluster'] == 3]
print(testcluster3.shape)

testcluster4 = testdata.loc[testdata['cluster'] == 4]
print(testcluster4.shape)

testcluster5 = testdata.loc[testdata['cluster'] == 5]
print(testcluster5.shape)

testcluster6 = testdata.loc[testdata['cluster'] == 6]
print(testcluster6.shape)

testcluster7 = testdata.loc[testdata['cluster'] == 7]
print(testcluster7.shape)

(35005, 31)
(390809, 31)
(10110, 31)
(5249, 31)
(11629, 31)
(2108, 31)
(17531, 31)
(42929, 31)


# Cluster analysis

In [13]:
# Calinski Harabasz score
from sklearn import metrics
from sklearn.metrics import pairwise_distances

labels_train = data_outer['cluster']
X_train = dataset_for_clustering
labels_test = testdata['cluster']
X_test = testdata_for_clustering

print(metrics.calinski_harabasz_score(X_train, labels_train))
print(metrics.calinski_harabasz_score(X_test, labels_test))

86490.53844844054
21205.316400085325


In [14]:
# Davies Bouldin score
from sklearn.metrics import davies_bouldin_score

print(davies_bouldin_score(X_train, labels_train))
print(davies_bouldin_score(X_test, labels_test))

2.7378781055311334
2.7491964831124642


# 4. Matrix factorization

In [15]:
weight_c0 = cluster0.shape[0] / data_outer.shape[0]
weight_c1 = cluster1.shape[0] / data_outer.shape[0]
weight_c2 = cluster2.shape[0] / data_outer.shape[0]
weight_c3 = cluster3.shape[0] / data_outer.shape[0]
weight_c4 = cluster4.shape[0] / data_outer.shape[0]
weight_c5 = cluster5.shape[0] / data_outer.shape[0]
weight_c6 = cluster6.shape[0] / data_outer.shape[0]
weight_c7 = cluster7.shape[0] / data_outer.shape[0]

weight_tc0 = testcluster0.shape[0] / testdata.shape[0]
weight_tc1 = testcluster1.shape[0] / testdata.shape[0]
weight_tc2 = testcluster2.shape[0] / testdata.shape[0]
weight_tc3 = testcluster3.shape[0] / testdata.shape[0]
weight_tc4 = testcluster4.shape[0] / testdata.shape[0]
weight_tc5 = testcluster5.shape[0] / testdata.shape[0]
weight_tc6 = testcluster6.shape[0] / testdata.shape[0]
weight_tc7 = testcluster7.shape[0] / testdata.shape[0]

# 5. KNN

#### Without cross validation

In [16]:
sim_options = {'name': 'cosine',
              'user_based': False}

model_knn = KNNBasic(sim_options=sim_options, k=16, verbose=True)

In [ ]:
# the optimal k=18 (hyperparamter tuning in 'feature transformation' notebook)

In [17]:
reader = Reader(line_format = 'user item rating', rating_scale=(0, 35))
cluster0 = Dataset.load_from_df(cluster0[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster1 = Dataset.load_from_df(cluster1[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster2 = Dataset.load_from_df(cluster2[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster3 = Dataset.load_from_df(cluster3[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster4 = Dataset.load_from_df(cluster4[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster5 = Dataset.load_from_df(cluster5[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster6 = Dataset.load_from_df(cluster6[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster7 = Dataset.load_from_df(cluster7[['user_id', 'song_id', 'rel_rating']], reader = reader)

type(cluster0)

surprise.dataset.DatasetAutoFolds

In [18]:
# split data into train and validation data. so now we have train, validation and test data.
from surprise.model_selection import train_test_split

train_cluster0, val_cluster0 = train_test_split(cluster0, test_size=0.25, random_state=42)
train_cluster1, val_cluster1 = train_test_split(cluster1, test_size=0.25, random_state=42)
train_cluster2, val_cluster2 = train_test_split(cluster2, test_size=0.25, random_state=42)
train_cluster3, val_cluster3 = train_test_split(cluster3, test_size=0.25, random_state=42)
train_cluster4, val_cluster4 = train_test_split(cluster4, test_size=0.25, random_state=42)
train_cluster5, val_cluster5 = train_test_split(cluster5, test_size=0.25, random_state=42)
train_cluster6, val_cluster6 = train_test_split(cluster6, test_size=0.25, random_state=42)
train_cluster7, val_cluster7 = train_test_split(cluster7, test_size=0.25, random_state=42)

print(type(train_cluster1), type(val_cluster1))

<class 'surprise.trainset.Trainset'> <class 'list'>


In [19]:
# get train set in a different format
exc00, newtrain_cluster0 = train_test_split(cluster0, test_size=0.999, random_state=42)
exc11, newtrain_cluster1 = train_test_split(cluster1, test_size=0.999, random_state=42)
exc22, newtrain_cluster2 = train_test_split(cluster2, test_size=0.999, random_state=42)
exc33, newtrain_cluster3 = train_test_split(cluster3, test_size=0.999, random_state=42)
exc44, newtrain_cluster4 = train_test_split(cluster4, test_size=0.999, random_state=42)
exc55, newtrain_cluster5 = train_test_split(cluster5, test_size=0.999, random_state=42)
exc66, newtrain_cluster6 = train_test_split(cluster6, test_size=0.999, random_state=42)
exc77, newtrain_cluster7 = train_test_split(cluster7, test_size=0.999, random_state=42)
print(type(exc00), type(train_cluster0))

<class 'surprise.trainset.Trainset'> <class 'surprise.trainset.Trainset'>


In [16]:
# change test data to the surprise format

In [20]:
reader = Reader(line_format = 'user item rating', rating_scale=(0, 35))
test_cluster0 = Dataset.load_from_df(testcluster0[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster1 = Dataset.load_from_df(testcluster1[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster2 = Dataset.load_from_df(testcluster2[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster3 = Dataset.load_from_df(testcluster3[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster4 = Dataset.load_from_df(testcluster4[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster5 = Dataset.load_from_df(testcluster5[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster6 = Dataset.load_from_df(testcluster6[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster7 = Dataset.load_from_df(testcluster7[['user_id', 'song_id', 'rel_rating']], reader = reader)

type(test_cluster1)

surprise.dataset.DatasetAutoFolds

In [21]:
# split data into test and excess dataset to get a different type
from surprise.model_selection import train_test_split

exc0, test_cluster0 = train_test_split(test_cluster0, test_size=0.999, random_state=42)
exc1, test_cluster1 = train_test_split(test_cluster1, test_size=0.999, random_state=42)
exc2, test_cluster2 = train_test_split(test_cluster2, test_size=0.999, random_state=42)
exc3, test_cluster3 = train_test_split(test_cluster3, test_size=0.999, random_state=42)
exc4, test_cluster4 = train_test_split(test_cluster4, test_size=0.999, random_state=42)
exc5, test_cluster5 = train_test_split(test_cluster5, test_size=0.999, random_state=42)
exc6, test_cluster6 = train_test_split(test_cluster6, test_size=0.999, random_state=42)
exc7, test_cluster7 = train_test_split(test_cluster7, test_size=0.999, random_state=42)

print(type(exc1), type(test_cluster1))

<class 'surprise.trainset.Trainset'> <class 'list'>


In [22]:
# find the train mae
prediction0_0 = model_knn.fit(train_cluster0).test(newtrain_cluster0)
prediction1_0 = model_knn.fit(train_cluster1).test(newtrain_cluster1)
prediction2_0 = model_knn.fit(train_cluster2).test(newtrain_cluster2)
prediction3_0 = model_knn.fit(train_cluster3).test(newtrain_cluster3)
prediction4_0 = model_knn.fit(train_cluster4).test(newtrain_cluster4)
prediction5_0 = model_knn.fit(train_cluster5).test(newtrain_cluster5)
prediction6_0 = model_knn.fit(train_cluster6).test(newtrain_cluster6)
prediction7_0 = model_knn.fit(train_cluster7).test(newtrain_cluster7)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [23]:
# find the validation set mae
prediction0_1 = model_knn.fit(train_cluster0).test(val_cluster0)
prediction1_1 = model_knn.fit(train_cluster1).test(val_cluster1)
prediction2_1 = model_knn.fit(train_cluster2).test(val_cluster2)
prediction3_1 = model_knn.fit(train_cluster3).test(val_cluster3)
prediction4_1 = model_knn.fit(train_cluster4).test(val_cluster4)
prediction5_1 = model_knn.fit(train_cluster5).test(val_cluster5)
prediction6_1 = model_knn.fit(train_cluster6).test(val_cluster6)
prediction7_1 = model_knn.fit(train_cluster7).test(val_cluster7)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [24]:
# check mae for test set for model comparison
prediction0_2 = model_knn.fit(train_cluster0).test(test_cluster0)
prediction1_2 = model_knn.fit(train_cluster1).test(test_cluster1)
prediction2_2 = model_knn.fit(train_cluster2).test(test_cluster2)
prediction3_2 = model_knn.fit(train_cluster3).test(test_cluster3)
prediction4_2 = model_knn.fit(train_cluster4).test(test_cluster4)
prediction5_2 = model_knn.fit(train_cluster5).test(test_cluster5)
prediction6_2 = model_knn.fit(train_cluster6).test(test_cluster6)
prediction7_2 = model_knn.fit(train_cluster7).test(test_cluster7)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [23]:
# TRAIN DATA
result0 = pd.DataFrame(prediction0_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

result4 = pd.DataFrame(prediction4_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result4.drop(columns = {'details'}, inplace=True)
result4['error'] = abs(result4['base_event'] - result4['predict_event'])
result4.head()

result5 = pd.DataFrame(prediction5_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result5.drop(columns = {'details'}, inplace=True)
result5['error'] = abs(result5['base_event'] - result5['predict_event'])
result5.head()

result6 = pd.DataFrame(prediction6_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result6.drop(columns = {'details'}, inplace=True)
result6['error'] = abs(result6['base_event'] - result6['predict_event'])
result6.head()

result7 = pd.DataFrame(prediction7_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result7.drop(columns = {'details'}, inplace=True)
result7['error'] = abs(result7['base_event'] - result7['predict_event'])
result7.head()

,visitor_id,item_id,base_event,predict_event,error
0,8721d2ed97a2e548229c5e7e89e08ac873fc0172,SOGRPEY12A8C13FBB1,1.790541,1.273157,0.517383
1,39004b3aa58282c20de9d435207f13bae3ebf019,SOLOHIX12A58A7A2C2,0.571429,0.571429,0.000000
2,57e947933a75ec6e7c85c78f1c1a466790414999,SORZCRI12A8AE4807B,0.435294,0.818859,0.383565
3,780fc48ec0ce06e4078f7f7560a502db516276c9,SOPJULC12A6D4F667F,2.802198,1.918909,0.883289
4,9736e1a5af0b4227e167d28ddf85ab538379fb1f,SOHGWFC12AB017F2E7,2.270270,1.445591,0.824679


In [25]:
# VALIDATION
result0 = pd.DataFrame(prediction0_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

result4 = pd.DataFrame(prediction4_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result4.drop(columns = {'details'}, inplace=True)
result4['error'] = abs(result4['base_event'] - result4['predict_event'])
result4.head()

result5 = pd.DataFrame(prediction5_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result5.drop(columns = {'details'}, inplace=True)
result5['error'] = abs(result5['base_event'] - result5['predict_event'])
result5.head()

result6 = pd.DataFrame(prediction6_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result6.drop(columns = {'details'}, inplace=True)
result6['error'] = abs(result6['base_event'] - result6['predict_event'])
result6.head()

result7 = pd.DataFrame(prediction7_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result7.drop(columns = {'details'}, inplace=True)
result7['error'] = abs(result7['base_event'] - result7['predict_event'])
result7.head()

,visitor_id,item_id,base_event,predict_event,error
0,df0683724c5e678af0b3e8a7098ebcb4feb01866,SODRFRJ12A8C144167,0.978723,0.952519,0.026205
1,a18bf764b4664d6e4d254f26cae88b87b2bfd882,SOWRQYD12AF729FD3A,0.486486,2.000980,1.514493
2,9bd2c25ff84e4a97198c484fbeb01d035739f3ab,SOHDWWH12A6D4F7F6A,0.233010,0.453474,0.220464
3,071a964898dc8f6d4aef983fa45bbea65ab29168,SOISNSU12AC468C0D8,0.168067,0.437096,0.269028
4,ee21c2dda11dc37a54d1c3b9ded8ab25622864e9,SOZZEID12A58A7AB89,0.610738,0.610738,0.000000


In [58]:
# TEST
result0 = pd.DataFrame(prediction0_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

result4 = pd.DataFrame(prediction4_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result4.drop(columns = {'details'}, inplace=True)
result4['error'] = abs(result4['base_event'] - result4['predict_event'])
result4.head()

result5 = pd.DataFrame(prediction5_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result5.drop(columns = {'details'}, inplace=True)
result5['error'] = abs(result5['base_event'] - result5['predict_event'])
result5.head()

result6 = pd.DataFrame(prediction6_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result6.drop(columns = {'details'}, inplace=True)
result6['error'] = abs(result6['base_event'] - result6['predict_event'])
result6.head()

result7 = pd.DataFrame(prediction7_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result7.drop(columns = {'details'}, inplace=True)
result7['error'] = abs(result7['base_event'] - result7['predict_event'])
result7.head()

,visitor_id,item_id,base_event,predict_event,error
0,e68c1f4c057f6dcc443db30deaed0cb970a4515d,SOLGWLZ12A6701C7A1,0.428571,0.731822,0.303251
1,2fc7b13f373c02c76be67c5a9f2739f6ceecacd2,SOQUOPM12A6D4F841F,0.662791,1.102009,0.439219
2,7f0cecc63f9291ea7edca69307114f32f9d5860d,SOLZEED12A6D4FA8C9,1.531915,1.038840,0.493075
3,89dff344978b815a092f92b46fa40588f3dad8a1,SOJTUED12AF729F915,0.900433,1.066687,0.166254
4,632bfc7a6f51bea09669fbf6dc04a381a4a9958d,SOCGWCK12A67020F39,0.368421,0.505967,0.137546


In [59]:
play_count_categories = []

for number in result0['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result0 = result0.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result0['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result0 = result0.assign(predict_binary = play_count_categories1)

result0.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,61953c2d342b1c2a4783c90f666472092b7e3b39,SORLZXM12A67ADAEF5,1.172414,0.706645,0.465769,1,0
1,c6d7f1fc78b052ea7bfd4056307c238ac1e5e3f2,SOCAIKQ12A6701E578,1.016949,0.707008,0.309941,1,0
2,0c27e3afce65e22540314d857fabc7ea7cd3d442,SOWGEWN12A6D4F789B,1.037344,0.518672,0.518672,1,0
3,2012e76622852e4c904c7555300bc361648b77ee,SOWYYUQ12A6701D68D,0.859873,1.198109,0.338236,0,1
4,7fea92ecaaf0e20b0d15785e2edd3c1b51de9387,SOPXKYD12A6D4FA876,0.736842,0.736842,0.000000,0,0


In [60]:
play_count_categories = []

for number in result1['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result1 = result1.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result1['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result1 = result1.assign(predict_binary = play_count_categories1)

result1.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,4ace65a52f0bc0e34dfd6b4140a555133f84b045,SOWEFXC12A6D4FA230,1.061728,0.896597,0.165131,1,0
1,0847b40aaba9cd33a289a9fea0abdee8595d7591,SOTRSFZ12A8C142BF6,0.600000,1.098182,0.498182,0,1
2,36755932b12ba31f8747e6eb22dd10ff8214ac2a,SOJSXJY12A8C13E32E,0.818182,0.818182,0.000000,0,0
3,c6112fc052d387397569ac77f0eb512720476213,SOHMXQJ12A6D4F77E0,0.784314,1.136758,0.352444,0,1
4,93f4bbf8b7a19d5c77609204501d77c85956ba3f,SOIZFTE12AB0186842,0.443038,0.701018,0.257980,0,0


In [61]:
play_count_categories = []

for number in result2['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result2 = result2.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result2['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result2 = result2.assign(predict_binary = play_count_categories1)

result2.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,a34c6288d3953fe656ede1555b2d7277dc6a3086,SOQGVCS12AF72A078D,0.914956,0.718261,0.196695,0,0
1,9d52659feb0887bb48101b60332f389bd0762e14,SOKJRAT12A8C142ECA,0.714286,0.714286,0.000000,0,0
2,ba993ccb4baa8544491af68db3ddb4175769c65a,SODABFB12A58A81788,0.292857,0.638054,0.345197,0,0
3,d9918e4b8160cbc50bf7dd358614b138e20d40b4,SOVGLOC12A8AE46F8D,3.524590,1.644809,1.879781,1,1
4,23c7d52bacd91b8a77601fb47c6e36a13559a927,SOMFBHU12A679AE567,0.500000,0.500000,0.000000,0,0


In [62]:
play_count_categories = []

for number in result3['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result3 = result3.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result3['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result3 = result3.assign(predict_binary = play_count_categories1)

result3.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,5e53383f4b4be3d4ea87cc34bf5d4880187da337,SOFGJCW12AF72A812D,0.723810,1.013779,0.289969,0,1
1,9c2e9434fadb0ccb5d1ce0e512c8bc8450fc3e2c,SOFGJCW12AF72A812D,0.894737,0.894737,0.000000,0,0
2,03f7e4ac0d94229253aa4c5f6801e23d268ba1aa,SOFGJCW12AF72A812D,0.532033,0.678783,0.146749,0,0
3,8f8f909dd86e77969d1f8753e10ac56c4c68aa72,SOHRZXH12A6D4FAB1A,3.255814,3.255814,0.000000,1,1
4,f43f0a720a04a9bc337a7494811c1d0a4901b3ca,SOHXWUX12AB01821DC,0.142857,0.142857,0.000000,0,0


In [63]:
play_count_categories = []

for number in result4['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result4 = result4.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result4['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result4 = result4.assign(predict_binary = play_count_categories1)

result4.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,493ed79d78c4e39de05623acc9a514f64c84b82e,SOUVTSM12AC468F6A7,1.400000,1.400000,0.000000,1,1
1,e8e8c93eee4d5e281ff2ea6cc92ef74f0eb1512b,SOZXVWK12AB018B865,1.147059,0.708433,0.438626,1,0
2,1d2341c337e43a82a4fc4d01f6646874508855fb,SOTBGRI12AF729D5A7,0.370920,0.698458,0.327539,0,0
3,8508f9ffa04fd2647cf4ed2636a2a3240f6a8daf,SOWCBKV12AC90732A6,0.705882,0.705882,0.000000,0,0
4,5f15708e8791afa2fed88fcac9a89acdb8f636d7,SOXTSFL12A81C217DD,0.781250,0.781250,0.000000,0,0


In [64]:
play_count_categories = []

for number in result5['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result5 = result5.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result5['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result5 = result5.assign(predict_binary = play_count_categories1)

result5.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,73411fe52391b2561cde6f57bd43f9f4f6555647,SODGVGW12AC9075A8D,0.808889,0.808889,0.000000,0,0
1,bc81d4e9fcd8f1d177695ce95fd09137ce0290a4,SOITRPB12A679AE5B2,0.500000,0.500000,0.000000,0,0
2,c35349d366d11d49f8cfa150eb3ca597cd61bdb4,SOJCIOE12A8C136594,0.844203,0.633152,0.211051,0,0
3,79c6877455baf5da1c7b0729623519bbecc61155,SOJLXET12AB0188632,3.562500,0.949779,2.612721,1,0
4,7da75612002261a0ca631e0037aee3474c484e3a,SODGVGW12AC9075A8D,0.884615,0.884615,0.000000,0,0


In [65]:
play_count_categories = []

for number in result6['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result6 = result6.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result6['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result6 = result6.assign(predict_binary = play_count_categories1)

result6.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,3631707f571f06ab67c4ea444f5885bd31c6a21d,SOESTZQ12A67021556,0.666667,1.306275,0.639608,0,1
1,ec104ac7fa393a04ad03c795180b5728d92b6ce2,SOXLRDB12A81C21739,0.785714,0.785714,0.000000,0,0
2,58123e3a5b24c3648d41fd55b733a2300d110e52,SOCLMAD12AB017FC09,0.926829,0.567068,0.359761,0,0
3,8ffc195c8df7f7280bc5e7a2707bb7689a0d646d,SOVSTAJ12A6D4F5F5E,0.565891,0.565891,0.000000,0,0
4,ce2f83cd4f4bdea2b3206fd56c20c8180c1eb5c1,SOGMAVD12A6D4F490C,1.766917,1.812358,0.045441,1,1


In [66]:
play_count_categories = []

for number in result7['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result7 = result7.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result7['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result7 = result7.assign(predict_binary = play_count_categories1)

result7.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,e68c1f4c057f6dcc443db30deaed0cb970a4515d,SOLGWLZ12A6701C7A1,0.428571,0.731822,0.303251,0,0
1,2fc7b13f373c02c76be67c5a9f2739f6ceecacd2,SOQUOPM12A6D4F841F,0.662791,1.102009,0.439219,0,1
2,7f0cecc63f9291ea7edca69307114f32f9d5860d,SOLZEED12A6D4FA8C9,1.531915,1.038840,0.493075,1,1
3,89dff344978b815a092f92b46fa40588f3dad8a1,SOJTUED12AF729F915,0.900433,1.066687,0.166254,0,1
4,632bfc7a6f51bea09669fbf6dc04a381a4a9958d,SOCGWCK12A67020F39,0.368421,0.505967,0.137546,0,0


In [67]:
# calculate accuracy
from sklearn.metrics import confusion_matrix

y_actual0 = pd.Series(result0['base_binary'], name='actual')
y_predict0 = pd.Series(result0['predict_binary'], name='predicted')
con_mat0 = (pd.crosstab(y_actual0, y_predict0)) / (pd.crosstab(y_actual0, y_predict0)).sum(axis=1)

y_actual1 = pd.Series(result1['base_binary'], name='actual')
y_predict1 = pd.Series(result1['predict_binary'], name='predicted')
con_mat1 = (pd.crosstab(y_actual1, y_predict1)) / (pd.crosstab(y_actual1, y_predict1)).sum(axis=1)

y_actual2 = pd.Series(result2['base_binary'], name='actual')
y_predict2 = pd.Series(result2['predict_binary'], name='predicted')
con_mat2 = (pd.crosstab(y_actual2, y_predict2)) / (pd.crosstab(y_actual2, y_predict2)).sum(axis=1)

y_actual3 = pd.Series(result3['base_binary'], name='actual')
y_predict3 = pd.Series(result3['predict_binary'], name='predicted')
con_mat3 = (pd.crosstab(y_actual3, y_predict3)) / (pd.crosstab(y_actual3, y_predict3)).sum(axis=1)

y_actual4 = pd.Series(result4['base_binary'], name='actual')
y_predict4 = pd.Series(result4['predict_binary'], name='predicted')
con_mat4 = (pd.crosstab(y_actual4, y_predict4)) / (pd.crosstab(y_actual4, y_predict4)).sum(axis=1)

In [68]:
y_actual5 = pd.Series(result5['base_binary'], name='actual')
y_predict5 = pd.Series(result5['predict_binary'], name='predicted')
con_mat5 = (pd.crosstab(y_actual5, y_predict5)) / (pd.crosstab(y_actual5, y_predict5)).sum(axis=1)

y_actual6 = pd.Series(result6['base_binary'], name='actual')
y_predict6 = pd.Series(result6['predict_binary'], name='predicted')
con_mat6 = (pd.crosstab(y_actual6, y_predict6)) / (pd.crosstab(y_actual6, y_predict6)).sum(axis=1)

y_actual7 = pd.Series(result7['base_binary'], name='actual')
y_predict7 = pd.Series(result7['predict_binary'], name='predicted')
con_mat7 = (pd.crosstab(y_actual7, y_predict7)) / (pd.crosstab(y_actual7, y_predict7)).sum(axis=1)

In [69]:
from sklearn import metrics

accuracy0 = metrics.accuracy_score(y_actual0, y_predict0)
accuracy1 = metrics.accuracy_score(y_actual1, y_predict1)
accuracy2 = metrics.accuracy_score(y_actual2, y_predict2)
accuracy3 = metrics.accuracy_score(y_actual3, y_predict3)
accuracy4 = metrics.accuracy_score(y_actual4, y_predict4)
accuracy5 = metrics.accuracy_score(y_actual5, y_predict5)
accuracy6 = metrics.accuracy_score(y_actual6, y_predict6)
accuracy7 = metrics.accuracy_score(y_actual7, y_predict7)
accuracy = accuracy0*weight_tc0 + accuracy1*weight_tc1 + accuracy2*weight_tc2 + accuracy3*weight_tc3 + accuracy4*weight_tc4 + accuracy5*weight_tc5 + accuracy6*weight_tc6 + accuracy7*weight_tc7

print('\nAccuracy:', round(accuracy,3))


Accuracy: 0.677


In [70]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# NOG VAN APARTE CLUSTERS MAKEN
conmat0 = confusion_matrix(y_actual0, y_predict0)
conmat1 = confusion_matrix(y_actual1, y_predict1)
conmat2 = confusion_matrix(y_actual2, y_predict2)
conmat3 = confusion_matrix(y_actual3, y_predict3)
conmat4 = confusion_matrix(y_actual4, y_predict4)
conmat5 = confusion_matrix(y_actual5, y_predict5)
conmat6 = confusion_matrix(y_actual6, y_predict6)
conmat7 = confusion_matrix(y_actual7, y_predict7)

TP = (con_mat0[1][1])*weight_tc0 + (con_mat1[1][1])*weight_tc1 + (con_mat2[1][1])*weight_tc2 + (con_mat3[1][1])*weight_tc3 + (con_mat4[1][1])*weight_tc4 + (con_mat5[1][1])*weight_tc5 + (con_mat6[1][1])*weight_tc6 + (con_mat7[1][1])*weight_tc7 
TN = (con_mat0[0][0])*weight_tc0 + (con_mat1[0][0])*weight_tc1 + (con_mat2[0][0])*weight_tc2 + (con_mat3[0][0])*weight_tc3 + (con_mat4[0][0])*weight_tc4 + (con_mat5[0][0])*weight_tc5 + (con_mat6[0][0])*weight_tc6 + (con_mat7[0][0])*weight_tc7 
FP = (con_mat0[0][1])*weight_tc0 + (con_mat1[0][1])*weight_tc1 + (con_mat2[0][1])*weight_tc2 + (con_mat3[0][1])*weight_tc3 + (con_mat4[0][1])*weight_tc4 + (con_mat5[0][1])*weight_tc5 + (con_mat6[0][1])*weight_tc6 + (con_mat7[0][1])*weight_tc7
FN = (con_mat0[1][0])*weight_tc0 + (con_mat1[1][0])*weight_tc1 + (con_mat2[1][0])*weight_tc2 + (con_mat3[1][0])*weight_tc3 + (con_mat4[1][0])*weight_tc4 + (con_mat5[1][0])*weight_tc5 + (con_mat6[1][0])*weight_tc6 + (con_mat7[1][0])*weight_tc7

print('TP:',TP,'\nTN:',TN,'\nFP:',FP,'\nFN:',FN)

TPshare = TP/(TP + TN + FP + FN)
TNshare = TN/(TP + TN + FP + FN)
FPshare = FP/(TP + TN + FP + FN)
FNshare = FN/(TP + TN + FP + FN)
print('TP', TPshare, '\nTN', TNshare, '\nFP', FPshare, '\nFN', FNshare)

TP: 0.6264449332551388 
TN: 0.6991856726853602 
FP: 0.16053333444898873 
FN: 0.6987711804210254
TP 0.28671099992330934 
TN 0.3200029447217608 
FP 0.07347281524287919 
FN 0.3198132401120508


In [73]:
print(conmat0)
print(conmat1)
print(conmat2)
print(conmat3)
print(conmat4)
print(conmat5)
print(conmat6)
print(conmat7)

[[19574  4666]
 [ 3391  7339]]
[[179461  93389]
 [ 46274  71295]]
[[5330 1623]
 [ 475 2672]]
[[3492  220]
 [ 450 1082]]
[[7415  860]
 [1018 2325]]
[[1467   17]
 [  66  556]]
[[10394  1794]
 [ 1722  3604]]
[[24689  5694]
 [ 4409  8095]]


In [72]:
ttl_TP = conmat0[1][1] + conmat1[1][1] + conmat2[1][1] + conmat3[1][1] + conmat5[1][1] + conmat4[1][1] + conmat6[1][1] + conmat7[1][1]
ttl_TN = conmat0[0][0] + conmat1[0][0] + conmat2[0][0] + conmat3[0][0]+ conmat4[0][0] + conmat5[0][0] + conmat6[0][0] + conmat7[0][0]
ttl_FP = conmat0[0][1] + conmat1[0][1] + conmat2[0][1] + conmat3[0][1]+ conmat4[0][1] + conmat5[0][1] + conmat6[0][1] + conmat7[0][1]
ttl_FN = conmat0[1][0] + conmat1[1][0] + conmat2[1][0] + conmat3[1][0]+ conmat4[1][0] + conmat5[1][0] + conmat6[1][0] + conmat7[1][0]

print(ttl_TP, ttl_TN, ttl_FP, ttl_FN)

96968 251822 108263 57805


In [ ]:
"""[[19574  4666]
 [ 3391  7339]]
[[179461  93389]
 [ 46274  71295]]
[[5330 1623]
 [ 475 2672]]
[[3492  220]
 [ 450 1082]]
[[7415  860]
 [1018 2325]]
[[1467   17]
 [  66  556]]
[[10394  1794]
 [ 1722  3604]]
 [[24689  5694]
 [ 4409  8095]]"""